# Data Engineer Challenge - Análisis de Tweets

Este notebook contiene la explicación detallada de las soluciones implementadas para el desafío de ingeniería de datos.

## Estructura del Challenge

Se deben resolver 3 problemas con 2 enfoques cada uno:
1. **Q1**: Top 10 fechas con más tweets + usuario más activo por fecha
2. **Q2**: Top 10 emojis más usados
3. **Q3**: Top 10 usuarios más influyentes (por menciones recibidas)

Cada problema tiene dos versiones:
- **`*_time`**: Optimizado para tiempo de ejecución
- **`*_memory`**: Optimizado para uso de memoria

## Supuestos y Consideraciones

1. **Formato de datos**: El archivo es JSONL (JSON Lines), una línea por tweet
2. **Estructura del tweet**: 
   - Campo `date`: fecha en formato ISO (ej: "2021-02-04T10:30:00Z")
   - Campo `user.username`: nombre de usuario del autor
   - Campo `content`: texto del tweet
3. **Manejo de errores**: Se ignoran líneas mal formateadas o con datos faltantes
4. **Encoding**: UTF-8 para soportar emojis y caracteres especiales
5. **Menciones**: Se detectan con regex `@(\w+)` (solo alfanuméricos y guiones bajos)

In [ ]:
# Configuración inicial
import sys
import time
from memory_profiler import profile
import json

# Ruta al archivo de datos
file_path = "../farmers-protest-tweets-2021-2-4.json"

# Imports de las funciones
from q1_time import q1_time
from q1_memory import q1_memory
from q2_time import q2_time
from q2_memory import q2_memory
from q3_time import q3_time
from q3_memory import q3_memory

print("✅ Imports completados")

## Verificación del archivo de datos

Primero verificamos que el archivo existe y vemos una muestra de su estructura.

In [ ]:
# Verificar que el archivo existe
import os
if os.path.exists(file_path):
    file_size = os.path.getsize(file_path) / (1024 * 1024)  # MB
    print(f"✅ Archivo encontrado: {file_path}")
    print(f"📊 Tamaño: {file_size:.2f} MB")
    
    # Leer primera línea como muestra
    with open(file_path, 'r', encoding='utf-8') as f:
        first_line = f.readline()
        sample_tweet = json.loads(first_line)
        print("\n📝 Estructura del tweet (primeros campos):")
        for key in list(sample_tweet.keys())[:10]:
            print(f"  - {key}: {type(sample_tweet[key]).__name__}")
else:
    print(f"❌ Archivo no encontrado: {file_path}")

## Q1: Top 10 fechas con más tweets + usuario más activo

### Estrategia `q1_time`:
- Carga todo el archivo en memoria
- Usa `defaultdict` para contadores anidados (fecha → username → conteo)
- Dos estructuras: `date_counts` y `date_user_counts`
- Al final ordena y obtiene top 10

### Estrategia `q1_memory`:
- Procesa línea por línea (mismo enfoque que time en este caso)
- Mantiene solo contadores en memoria, no carga tweets completos
- Nota: Para archivos muy grandes, se podría optimizar más usando streaming y descartando datos intermedios

In [ ]:
# Prueba Q1 - Versión optimizada para tiempo
print("🕐 Ejecutando q1_time...")
start_time = time.time()
result_q1_time = q1_time(file_path)
time_q1_time = time.time() - start_time
print(f"⏱️  Tiempo: {time_q1_time:.2f} segundos")
print(f"📊 Resultados (primeros 3):")
for i, (date, username) in enumerate(result_q1_time[:3], 1):
    print(f"  {i}. {date} - {username}")
print(f"\n✅ Total resultados: {len(result_q1_time)}")

In [ ]:
# Prueba Q1 - Versión optimizada para memoria
print("💾 Ejecutando q1_memory...")
start_time = time.time()
result_q1_memory = q1_memory(file_path)
time_q1_memory = time.time() - start_time
print(f"⏱️  Tiempo: {time_q1_memory:.2f} segundos")
print(f"📊 Resultados (primeros 3):")
for i, (date, username) in enumerate(result_q1_memory[:3], 1):
    print(f"  {i}. {date} - {username}")
print(f"\n✅ Total resultados: {len(result_q1_memory)}")

# Verificar que los resultados coinciden
if result_q1_time == result_q1_memory:
    print("\n✅ Los resultados coinciden entre ambas versiones")
else:
    print("\n⚠️  Los resultados difieren (verificar implementación)")

## Q2: Top 10 emojis más usados

### Estrategia:
- Usa regex para detectar emojis Unicode en rangos específicos
- Patrón compilado una vez para eficiencia
- `Counter` para acumular conteos
- Ambas versiones (`time` y `memory`) son similares porque procesan línea por línea

### Rangos Unicode cubiertos:
- Emoticons (😀-🙏)
- Símbolos y pictogramas (🌀-🗿)
- Transporte y mapas (🚀-🛿)
- Banderas (🇦-🇿)
- Y otros rangos comunes de emojis

In [ ]:
# Prueba Q2 - Versión optimizada para tiempo
print("🕐 Ejecutando q2_time...")
start_time = time.time()
result_q2_time = q2_time(file_path)
time_q2_time = time.time() - start_time
print(f"⏱️  Tiempo: {time_q2_time:.2f} segundos")
print(f"📊 Top 10 emojis:")
for i, (emoji, count) in enumerate(result_q2_time, 1):
    print(f"  {i}. {emoji} - {count:,} veces")

In [ ]:
# Prueba Q2 - Versión optimizada para memoria
print("💾 Ejecutando q2_memory...")
start_time = time.time()
result_q2_memory = q2_memory(file_path)
time_q2_memory = time.time() - start_time
print(f"⏱️  Tiempo: {time_q2_memory:.2f} segundos")
print(f"📊 Top 10 emojis:")
for i, (emoji, count) in enumerate(result_q2_memory, 1):
    print(f"  {i}. {emoji} - {count:,} veces")

# Verificar que los resultados coinciden
if result_q2_time == result_q2_memory:
    print("\n✅ Los resultados coinciden entre ambas versiones")
else:
    print("\n⚠️  Los resultados difieren (verificar implementación)")

## Q3: Top 10 usuarios más influyentes (por menciones)

### Estrategia:
- Usa regex `@(\w+)` para detectar menciones en el texto
- `\w+` captura solo caracteres alfanuméricos y guiones bajos (estándar de Twitter)
- `Counter` acumula menciones por username
- Ambas versiones procesan línea por línea

### Nota sobre influencia:
La influencia se mide por el número de veces que un usuario es mencionado (@username) en los tweets, no por sus propios tweets.

In [ ]:
# Prueba Q3 - Versión optimizada para tiempo
print("🕐 Ejecutando q3_time...")
start_time = time.time()
result_q3_time = q3_time(file_path)
time_q3_time = time.time() - start_time
print(f"⏱️  Tiempo: {time_q3_time:.2f} segundos")
print(f"📊 Top 10 usuarios más mencionados:")
for i, (username, count) in enumerate(result_q3_time, 1):
    print(f"  {i}. @{username} - {count:,} menciones")

In [ ]:
# Prueba Q3 - Versión optimizada para memoria
print("💾 Ejecutando q3_memory...")
start_time = time.time()
result_q3_memory = q3_memory(file_path)
time_q3_memory = time.time() - start_time
print(f"⏱️  Tiempo: {time_q3_memory:.2f} segundos")
print(f"📊 Top 10 usuarios más mencionados:")
for i, (username, count) in enumerate(result_q3_memory, 1):
    print(f"  {i}. @{username} - {count:,} menciones")

# Verificar que los resultados coinciden
if result_q3_time == result_q3_memory:
    print("\n✅ Los resultados coinciden entre ambas versiones")
else:
    print("\n⚠️  Los resultados difieren (verificar implementación)")

## Resumen de Tiempos de Ejecución

Comparación entre versiones optimizadas para tiempo vs memoria:

In [ ]:
# Resumen comparativo
import pandas as pd

summary_data = {
    'Función': ['Q1', 'Q2', 'Q3'],
    'Tiempo (time)': [time_q1_time, time_q2_time, time_q3_time],
    'Tiempo (memory)': [time_q1_memory, time_q2_memory, time_q3_memory],
}

df_summary = pd.DataFrame(summary_data)
df_summary['Diferencia (%)'] = ((df_summary['Tiempo (memory)'] - df_summary['Tiempo (time)']) / df_summary['Tiempo (time)'] * 100).round(2)

print("📊 Resumen de tiempos de ejecución:")
print(df_summary.to_string(index=False))
print("\n💡 Nota: Las diferencias pueden variar según el tamaño del archivo y recursos del sistema")

## Mejoras Futuras

### Q1:
- **Time**: Para archivos más grandes, usar multiprocessing para procesar chunks en paralelo
- **Memory**: Implementar streaming con límite de memoria fijo, descartando fechas fuera del top 10 durante el procesamiento

### Q2:
- Usar librería especializada como `emoji` para mejor detección de emojis
- Considerar emojis compuestos (skin tones, flags combinados)
- Cachear regex compilado fuera de la función

### Q3:
- Validar que los usernames mencionados existan en el dataset
- Considerar case-insensitive matching si es necesario
- Filtrar menciones a usuarios propios del autor del tweet

### General:
- Implementar logging para debugging
- Agregar validación de tipos más estricta
- Considerar usar `pandas` o `polars` para procesamiento más eficiente en algunos casos
- Implementar tests unitarios con datos de muestra

## Medición de Memoria (Opcional)

Para medir el uso de memoria en detalle, se puede usar `memory_profiler`:

```python
# Ejemplo de uso:
# %load_ext memory_profiler
# %mprun -f q1_memory q1_memory(file_path)
```

O usar `memray` para análisis más avanzado:
```bash
memray run -o profile.bin python -c "from q1_memory import q1_memory; q1_memory('file_path')"
memray table profile.bin
```